### Symbols in Market Watch

### Max Sharpe Weights

In [10]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("ICM_MT5_LOGIN"))
password = os.getenv("ICM_MT5_PASSWORD")
server = os.getenv("ICM_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "AUS200": "AUDUSD",        # AUD-denominated
    "UK100": "GBPUSD",         # GBP-denominated
    "DE40": "EURUSD",          # EUR-denominated
    "STOXX50": "EURUSD",       # EUR-denominated
    "JP225": "USDJPY",         # flip sign
    "F40": "EURUSD"            # EUR-denominated
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=84):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df

def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

def ewma_covariance(returns: pd.DataFrame, lambda_: float = 0.94) -> pd.DataFrame:
    """
    Compute EWMA covariance matrix (RiskMetrics style).
    returns: DataFrame of aligned returns (T × N)
    lambda_: decay factor (0.94 for daily)
    """
    r = returns.values
    T, N = r.shape

    # Initialize with sample covariance of first few observations
    init_window = min(5, T)
    cov = np.cov(r[:init_window].T)

    for t in range(init_window, T):
        x = r[t].reshape(-1, 1)
        cov = lambda_ * cov + (1.0 - lambda_) * (x @ x.T)

    return pd.DataFrame(cov, index=returns.columns, columns=returns.columns)

# -----------------------------
# Load factor signals
# -----------------------------
# ai_factor_signal_icm.csv:  asset \t factor_signal
expected_returns_df = pd.read_csv("ai_factor_signal_icm.csv", sep="\t", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (
    (expected_returns_df["factor_signal"] - mean_signal) / std_signal
)

# Convert to dict
expected_returns_dict = dict(
    zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"])
)

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}


fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    # If pair is quoted as USDXXX (e.g., USDJPY, USDHKD), flip sign to get XXXUSD log return
    if fx.startswith("USD"):
        lr = -lr  # log(1/x) = -log(x)

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns for all assets
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        # Align index and FX by date
        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        # USD-adjusted return: r_usd = r_index_local + r_fx
        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        # Already in USD terms
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------

returns = pd.DataFrame(asset_returns)[symbols].dropna(how="any")

# -----------------------------
# Compute daily volatilities
# -----------------------------
daily_vols = returns.std().values  # vector aligned with symbols

# -----------------------------
# EWMA Covariance matrix
# -----------------------------
lambda_ = 0.94
cov_matrix = ewma_covariance(returns, lambda_=lambda_)
cov_matrix = cov_matrix + 1e-6 * np.eye(len(cov_matrix))

# -----------------------------
# Expected daily returns vector (volatility-scaled)
# -----------------------------
IC_estimate = 0.05
zscores = np.array([expected_returns_dict[symbol] for symbol in symbols])

if IC_estimate < 0:
    zscores = -zscores
    IC_estimate = abs(IC_estimate)

# Multiply by volatility
expected_returns = IC_estimate * zscores * daily_vols

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        if port_vol <= 0:
            return 1e6
        return -(port_ret - risk_free_rate) / port_vol

    # REMOVE dollar-neutral constraint
    constraints = [
        {"type": "ineq", "fun": lambda w: 1.0 - np.sum(w**2)}  # L2 leverage only
    ]

    # Allow long/short
    bounds = [(-1, 1)] * num_assets

    # Initial guess: simple equal-weight (no neutrality enforcement)
    init_guess = np.ones(num_assets) / num_assets

    result = minimize(
        neg_sharpe,
        init_guess,
        bounds=bounds,
        constraints=constraints,
        method="SLSQP",
    )

    if not result.success:
        print("Warning: optimization did not fully converge:", result.message)

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol


# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "asset": symbols,
    "weight": weights
})
weights_df.to_csv("icm_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to icm_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics
# -----------------------------
annual_port_return = daily_port_return * 252
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------

print("\n--- MODEL Portfolio Statistics ---")
print(f"Daily Return: {daily_port_return:.6%}")
print(f"Annualized Return: {annual_port_return:0.2%}")
print(f"Daily Volatility: {daily_port_vol:.6%}")
print(f"Annualized Volatility: {annual_port_vol:.2%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


Weights exported to icm_optimal_portfolio_weights.csv

--- MODEL Portfolio Statistics ---
Daily Return: 0.085201%
Annualized Return: 21.47%
Daily Volatility: 0.215616%
Annualized Volatility: 3.42%
Annualized Sharpe Ratio: 6.27


True

### Volatility Targeted Weights using EWMA estimates

In [11]:
import pandas as pd
import numpy as np

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values
print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX (DAILY) ---
# ============================================================
lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# EWMA update: use r_t at time t
for t in range(1, len(rets)):
    r = rets[t].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA daily volatility
daily_portfolio_returns = returns @ weights
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)

# ============================================================
# --- BASE PORTFOLIO STATISTICS ---
# ============================================================

ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
ewma_annual_return = daily_portfolio_returns.mean() * 252

# Sharpe ratio (realized, ex-post)
ewma_sharpe = ewma_annual_return / ewma_annual_vol

print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"Daily Portfolio Return (realized): {daily_portfolio_returns.mean():0.2%}")
print(f"Annual Portfolio Return (realized): {ewma_annual_return:0.2%}")
print(f"Sharpe Ratio (realized): {ewma_sharpe:0.4f}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_annual_vol = 0.05  # e.g. 10% annual target

# Convert target annual vol to daily
target_daily_vol = target_annual_vol / np.sqrt(252)

# Scale factor in DAILY units
scale_factor = target_daily_vol / ewma_daily_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

# Sharpe ratio after scaling (realized)
scaled_sharpe = scaled_annual_return / scaled_annual_vol

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_annual_vol:.0%}) ---")
print(f"Factor: {scale_factor:.4f}")
print(f"Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")


print(f"\n--- EX-ANTE ---")
print(f"Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Annual Portfolio Return: {scaled_annual_return:0.2%}")
print(f"Sharpe Ratio: {scaled_sharpe:0.4f}")


# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("icm_scaled_weights.csv", index=False)
print("\nScaled weights exported to icm_scaled_weights.csv")

Loaded weights (aligned):
[-0.17257701 -0.11267181 -0.06898211  0.06653197 -0.33956635 -0.01699488
 -0.03196717  0.03616239 -0.08696051 -0.29618489 -0.04211348 -0.03080589
  0.00673801 -0.50232557 -0.15793877  0.05956961  0.32900791  0.52668754
  0.04779816  0.16798416  0.18462656]

Asset order:
['JP225', 'USDJPY', 'XBRUSD', 'XTIUSD', 'F40', 'XAGUSD', 'USDCHF', 'XAUUSD', 'EURUSD', 'AUDUSD', 'USDCAD', 'GBPUSD', 'XNGUSD', 'US500', 'DE40', 'NZDUSD', 'USTEC', 'STOXX50', 'AUS200', 'UK100', 'US30']

--- EWMA VOLATILITY FORECAST ---
Daily Volatility: 0.19%
Annual Volatility: 3.02%
Daily Portfolio Return (realized): 0.02%
Annual Portfolio Return (realized): 5.06%
Sharpe Ratio (realized): 1.6738

--- SCALED PORTFOLIO (Target Vol = 5%) ---
Factor: 1.6548
Daily Volatility (EWMA): 0.31%
Annual Volatility (EWMA): 5.00%

--- EX-ANTE ---
Daily Portfolio Return: 0.03%
Annual Portfolio Return: 8.37%
Sharpe Ratio: 1.6738

Scaled weights exported to icm_scaled_weights.csv


In [12]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------------
# Connect to MT5
# -----------------------------------
if not mt5.initialize():
    print("MT5 initialization failed")
    quit()

# -----------------------------------
# Exact sequence of symbols
# -----------------------------------
symbols = [
    "AUDUSD",
    "EURUSD",
    "GBPUSD",
    "NZDUSD",
    "USDJPY",
    "USDCHF",
    "USDCAD",
    "XAGUSD",
    "XAUUSD",
    "XNGUSD",
    "XTIUSD",
    "XBRUSD",
    "US500",
    "US30",
    "USTEC",
    "DE40",
    "JP225",
    "UK100",
    "STOXX50",
    "AUS200",
    "F40"
]

# -----------------------------------
# Fetch contract sizes
# -----------------------------------
records = []

for symbol in symbols:
    mt5.symbol_select(symbol, True)
    info = mt5.symbol_info(symbol)

    if info is None:
        records.append([symbol, None])
    else:
        records.append([symbol, info.trade_contract_size])

# -----------------------------------
# Create DataFrame and export to CSV
# -----------------------------------
df = pd.DataFrame(records, columns=["asset", "contract_size"])
df.to_csv("icm_contract_size.csv", index=False)

# -----------------------------------
# Shutdown MT5
# -----------------------------------
mt5.shutdown()

True

### Lot Sizing

In [13]:
import math
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 200000.90  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY", "USDCHF", "USDCAD"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "AUS200": "AUDUSD",        # AUD-denominated
    "UK100": "GBPUSD",         # GBP-denominated
    "DE40": "EURUSD",          # EUR-denominated
    "STOXX50": "EURUSD",       # EUR-denominated
    "JP225": "USDJPY",         # flip sign
    "F40": "EURUSD"            # EUR-denominated
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.bid

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("icm_contract_size.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()


# Merge contract sizes + weights
df = contract_df.merge(scaled_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# FETCH NET POSITIONS FROM MT5
# ============================

def fetch_net_positions():
    positions = mt5.positions_get()
    if positions is None:
        return {}

    net = {}

    for p in positions:
        symbol = p.symbol
        volume = p.volume if p.type == 0 else -p.volume  # BUY = +, SELL = -
        net[symbol] = net.get(symbol, 0) + volume

    return net

net_positions = fetch_net_positions()
df["current_holdings"] = pd.to_numeric(df["asset"].map(net_positions).fillna(0), errors="coerce")

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["scaled_weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    if pd.isna(weight) or pd.isna(contract_size):
        return None
    if contract_size == 0:
        return None

    # FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    if price is None or price == 0:
        return None

    # Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        if asset == "JP225":
            fx_rate = 1 / fx_rate
        price = price * fx_rate
        if price == 0:
            return None

    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator


# ============================
# CURRENT WEIGHT CALCULATION
# ============================
def compute_current_weight(row):
    asset = row["asset"]
    lot = row.get("current_holdings", 0)
    contract_size = row["contract_size"]
    price = row["latest_price"]

    if pd.isna(lot) or pd.isna(contract_size):
        return None
    if contract_size == 0:
        return None

    # FX-exempt assets
    if asset in FX_EXEMPT:
        position_value = lot * contract_size
        return position_value / EQUITY

    if price is None or price == 0:
        return None

    # Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        if asset == "JPN225":
            fx_rate = 1 / fx_rate
        price = price * fx_rate
        if price == 0:
            return None

    position_value = lot * price * contract_size
    return position_value / EQUITY


from decimal import Decimal, ROUND_HALF_UP
import math

def safe_quantize(x, places="0.0001"):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    return Decimal(str(x)).quantize(Decimal(places), rounding=ROUND_HALF_UP)

# Compute current weights
df["current_weight"] = df.apply(compute_current_weight, axis=1)
df["current_weight"] = df["current_weight"].apply(lambda x: safe_quantize(x, "0.0001"))

# Compute target lot sizes
df["target_lot_size"] = df.apply(compute_lot, axis=1)
df["target_lot_size"] = df["target_lot_size"].apply(lambda x: safe_quantize(x, "0.01"))

# ============================

df["target_lot_size"] = pd.to_numeric(df["target_lot_size"], errors="coerce")

df["difference"] = df["target_lot_size"] - df["current_holdings"]
df["difference"] = pd.to_numeric(df["difference"], errors="coerce").round(2)
df.rename(columns={"scaled_weight": "target_weight"}, inplace=True)
# ============================
# OUTPUT
# ============================
print(df[["asset", "latest_price", "current_holdings", "target_lot_size", "difference"]].dropna())
df.dropna().to_csv("icm_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_target_lot_size"] = df["target_lot_size"].abs()
df["abs_current_holdings"] = df["current_holdings"].abs()
gross_target_lot_size = df["abs_target_lot_size"].sum()
gross_current_holdings = df["abs_current_holdings"].sum()

print(f"\nGross Total Target Lot Size (absolute): {gross_target_lot_size:.2f}")
print(f"Gross Current Holdings (absolute): {gross_current_holdings:.2f}")

mt5.shutdown()

      asset  latest_price  current_holdings  target_lot_size  difference
0    AUDUSD       0.66829             -1.97            -1.47        0.50
1    EURUSD       1.15983             -0.42            -0.25        0.17
2    GBPUSD       1.33756             -0.29            -0.08        0.21
3    NZDUSD       0.57497              0.72             0.34       -0.38
4    USDJPY     158.11900             -0.48            -0.37        0.11
5    USDCHF       0.80284             -0.21            -0.11        0.10
6    USDCAD       1.39154             -0.19            -0.14        0.05
7    XAGUSD      90.05100             -0.14            -0.06        0.08
8    XAUUSD    4595.65000              0.06             0.03       -0.03
9    XNGUSD       3.19500              0.00             0.07        0.07
10   XTIUSD      59.33000              0.00             3.71        3.71
11   XBRUSD      64.05000              0.00            -3.56       -3.56
12    US500    6941.90000              0.00        

True

In [14]:
df

,asset,contract_size,target_weight,latest_price,current_holdings,current_weight,target_lot_size,difference,abs_target_lot_size,abs_current_holdings
0,AUDUSD,100000.0,-0.490121,0.66829,-1.97,-0.6583,-1.47,0.50,1.47,1.97
1,EURUSD,100000.0,-0.143901,1.15983,-0.42,-0.2436,-0.25,0.17,0.25,0.42
2,GBPUSD,100000.0,-0.050977,1.33756,-0.29,-0.1939,-0.08,0.21,0.08,0.29
3,NZDUSD,100000.0,0.098575,0.57497,0.72,0.2070,0.34,-0.38,0.34,0.72
4,USDJPY,100000.0,-0.186447,158.11900,-0.48,-0.2400,-0.37,0.11,0.37,0.48
5,USDCHF,100000.0,-0.052899,0.80284,-0.21,-0.1050,-0.11,0.10,0.11,0.21
6,USDCAD,100000.0,-0.069689,1.39154,-0.19,-0.0950,-0.14,0.05,0.14,0.19
7,XAGUSD,1000.0,-0.028123,90.05100,-0.14,-0.0630,-0.06,0.08,0.06,0.14
8,XAUUSD,100.0,0.059841,4595.65000,0.06,0.1379,0.03,-0.03,0.03,0.06
9,XNGUSD,10000.0,0.011150,3.19500,0.00,0.0000,0.07,0.07,0.07,0.00


In [15]:
# --- 1. Cross‑sectional dispersion across assets ---

# daily dispersion (std across assets for each day)
dispersion = returns.std(axis=1)

# convert to percentage strings
dispersion_pct = dispersion.apply(lambda x: f"{x:.2%}")
print("\n--- DAILY CROSS-SECTIONAL DISPERSION ---")
print(dispersion_pct.tail(10))


# --- 2. Align weights with returns ---

# convert weights df into a Series indexed by asset
current_weights = df.set_index('asset')['current_weight']

# align weights to returns columns (critical step)
current_weights = current_weights.reindex(returns.columns).astype(float)

# optional: check for mismatches
missing_in_weights = set(returns.columns) - set(current_weights.index)
missing_in_returns = set(current_weights.index) - set(returns.columns)
if missing_in_weights:
    print("Assets in returns but missing weights:", missing_in_weights)
if missing_in_returns:
    print("Assets in weights but missing returns:", missing_in_returns)


# --- 3. Compute weighted portfolio returns (ex‑post) ---

# multiply each column by its weight and sum across assets
today_port_return = (returns.tail(1) * current_weights)
real_time_vol = today_port_return.std(axis=1) * np.sqrt(252)



print(f"\ntoday's portfolio volatility (REAL-TIME): ")

print(f"{real_time_vol.iloc[0]:.2%}")


--- DAILY CROSS-SECTIONAL DISPERSION ---
time
2026-01-05    1.77%
2026-01-06    1.61%
2026-01-07    1.22%
2026-01-08    1.62%
2026-01-09    2.15%
2026-01-12    1.72%
2026-01-13    0.87%
2026-01-14    2.49%
2026-01-15    0.98%
2026-01-16    0.64%
dtype: object

today's portfolio volatility (REAL-TIME): 
0.81%


In [16]:
today_port_return

,JP225,USDJPY,XBRUSD,XTIUSD,F40,XAGUSD,USDCHF,XAUUSD,EURUSD,AUDUSD,...,GBPUSD,XNGUSD,US500,DE40,NZDUSD,USTEC,STOXX50,AUS200,UK100,US30
time,,,,,,,,,,,,,,,,,,,,,
2026-01-16,-0.0,0.000752,0.0,0.0,-0.0,0.001591,0.000052,-0.000602,0.000233,0.001515,...,0.000087,-0.0,-0.0,0.0,0.00027,-0.0,0.0,0.0,0.0,-0.0


In [17]:
returns.tail(1) #* current_weights

,JP225,USDJPY,XBRUSD,XTIUSD,F40,XAGUSD,USDCHF,XAUUSD,EURUSD,AUDUSD,...,GBPUSD,XNGUSD,US500,DE40,NZDUSD,USTEC,STOXX50,AUS200,UK100,US30
time,,,,,,,,,,,,,,,,,,,,,
2026-01-16,-0.003387,-0.003132,0.002501,0.000674,-0.00445,-0.025253,-0.000498,-0.004366,-0.000957,-0.002302,...,-0.000448,-0.013986,-0.000518,0.001895,0.001305,-0.000697,0.001051,0.002891,0.002736,-0.001447


In [18]:
# basic descriptive statistics
desc_stats = dispersion.describe()

# add skewness and kurtosis
desc_stats.loc["skewness"] = dispersion.skew()
desc_stats.loc["kurtosis"] = dispersion.kurtosis()

print(desc_stats)



count       54.000000
mean         0.012118
std          0.004917
min          0.003003
25%          0.008681
50%          0.010888
75%          0.016161
max          0.024901
skewness     0.751664
kurtosis     0.044675
dtype: float64


In [ ]:
# strategy.py

import pandas as pd
import MetaTrader5 as mt5

from brokers import get_broker
from mapper import build_fx_map
from dataloader import (
    load_symbols_from_csv,
    load_log_returns,
    save_metadata,
    load_contract_dataframe
)
from covariance import get_covariance
from expectedreturns import compute_expected_returns
from optimizer import optimize_portfolio
from risk import risk_analysis
from lotsizing import run_lot_sizing


# ============================================================
# MAIN STRATEGY PIPELINE
# ============================================================
def run_strategy(
    broker_name: str,
    active_symbols_csv: str,
    factor_signal_csv: str,
    vol_target: float,
    ewma_lambda: float,
    ic: float,
    lookback_days: int = 60
):
    """
    Full portfolio construction + lot sizing pipeline.

    Parameters
    ----------
    broker_name : str
        Name of broker (e.g., "icmarkets")
    active_symbols_csv : str
        CSV file containing list of tradable symbols
    factor_signal_csv : str
        CSV file containing factor signals
    vol_target : float
        Annual volatility target (e.g., 0.10)
    ewma_lambda : float
        EWMA decay factor for covariance
    ic : float
        Information coefficient for expected returns
    lookback_days : int
        Lookback window for returns

    Returns
    -------
    df : DataFrame
        Final lot sizing table
    gross_target : float
        Total absolute target lots
    gross_current : float
        Total absolute current lots
    """

    # --------------------------------------------------------
    # 1. Initialize broker + MT5
    # --------------------------------------------------------
    broker = get_broker(broker_name)
    broker.initialize()

    # --------------------------------------------------------
    # 2. Load symbols
    # --------------------------------------------------------
    symbols = load_symbols_from_csv(active_symbols_csv)

    # --------------------------------------------------------
    # 3. FX mapping
    # --------------------------------------------------------
    fx_map, fx_exempt = build_fx_map(
        source="csv",
        csv_path=active_symbols_csv
    )

    # --------------------------------------------------------
    # 4. Load returns (with DB caching)
    # --------------------------------------------------------
    returns = load_log_returns(
        symbols=symbols,
        fx_map=fx_map,
        broker_name=broker.name,
        lookback_days=lookback_days
    )

    # --------------------------------------------------------
    # 5. Save metadata (contract size, min volume)
    # --------------------------------------------------------
    save_metadata(symbols=symbols, broker_name=broker.name)

    # --------------------------------------------------------
    # 6. Covariance matrix
    # --------------------------------------------------------
    cov = get_covariance(returns, method="ewma", decay=ewma_lambda)

    # --------------------------------------------------------
    # 7. Expected returns
    # --------------------------------------------------------
    expected_returns = compute_expected_returns(
        signals=factor_signal_csv,
        returns=returns,
        ic=ic,
        vol_target=vol_target,
        vol_window=lookback_days
    )

    # --------------------------------------------------------
    # 8. Optimize portfolio (Max Sharpe)
    # --------------------------------------------------------
    weights, daily_ret, daily_vol = optimize_portfolio(
        expected_returns,
        cov,
        dollar_neutral=True
    )

    # --------------------------------------------------------
    # 9. Volatility targeting
    # --------------------------------------------------------
    result = risk_analysis(weights, returns)
    scaled_weights = result["scaled_weights"]

    scaled_df = pd.DataFrame({
        "asset": returns.columns,
        "scaled_weight": scaled_weights
    })

    # --------------------------------------------------------
    # 10. Load contract metadata + merge with weights
    # --------------------------------------------------------
    df = load_contract_dataframe(
        broker_name=broker.name,
        scaled_df=scaled_df
    )

    # --------------------------------------------------------
    # 11. Lot sizing
    # --------------------------------------------------------
    df, gross_target, gross_current = run_lot_sizing(
        df=df,
        fx_exempt=fx_exempt,
        index_fx_map=fx_map
    )

    # --------------------------------------------------------
    # 12. Shutdown MT5
    # --------------------------------------------------------
    mt5.shutdown()

    return df, gross_target, gross_current

Updating DB from 2026-01-16 to today...
No new MT5 data available.
Saved metadata to table 'icmarkets_metadata'
      asset  contract_size  min_volume  current_holdings  target_lot_size
0    EURUSD       100000.0        0.01             -0.42            -0.91
1    GBPUSD       100000.0        0.01             -0.29             0.25
2    USDCHF       100000.0        0.01             -0.21            -0.15
3    USDJPY       100000.0        0.01             -0.48            -0.18
4    USDCAD       100000.0        0.01             -0.19             0.47
5    AUDUSD       100000.0        0.01             -1.97            -1.59
6    NZDUSD       100000.0        0.01              0.72             2.50
7    AUS200            1.0        0.10              0.00            -4.84
8   STOXX50            1.0        0.10              0.00             9.38
9       F40            1.0        0.10              0.00            -3.64
10    JP225            1.0        1.00              0.00           -79.21


True

In [10]:
result

{'cov': array([[ 4.83517754e-06,  6.19357283e-06, -6.48168629e-06,
         -4.86317086e-06, -3.13449595e-06,  3.08443542e-06,
          5.46629657e-06,  4.96480641e-06,  3.79293557e-06,
          4.62619140e-06,  1.17182823e-06,  4.79038495e-06,
          1.68116405e-06,  5.15623441e-07, -1.24768485e-06,
          9.30379019e-06,  6.80534758e-06,  8.20748724e-06,
          1.75862846e-05,  9.34048723e-06,  5.39736964e-06],
        [ 6.19357283e-06,  1.20619909e-05, -9.29363901e-06,
         -8.03722187e-06, -4.08961185e-06,  6.12464582e-06,
          1.00618706e-05,  1.12973699e-05,  8.33690610e-06,
          7.58973270e-06,  1.04986502e-05,  1.11496637e-05,
          5.20995372e-06,  2.81857681e-06,  1.05058455e-06,
          2.56112074e-05,  1.46444563e-05,  1.52549390e-05,
          9.02144811e-06,  1.75989638e-05,  1.02323322e-05],
        [-6.48168629e-06, -9.29363901e-06,  1.12384909e-05,
          7.62419355e-06,  4.85422904e-06, -4.80852355e-06,
         -9.18272407e-06, -9.89